In [1]:
import glob
import numpy as np
import pandas as pd

MC_DATA_FOLDER = "../results/MC_data_1e7"
TOTAL_NUMBER_OF_ANNIHILATIONS = 10_000_000
TOTAL_NUMBER_OF_PHOTONS = TOTAL_NUMBER_OF_ANNIHILATIONS * 2

Rs = [5.0, 10.0, 15.0]

# Fixed detector cylinder geometry
R_DET_CM = 15.0
RING_LENGTH_CM = 20.0
Z_MAX_DET_CM = 0.5 * RING_LENGTH_CM

def pct(n, denom):
    return 100.0 * n / denom if denom else float("nan")

def latex_int(n: int) -> str:
    return f"{int(n):,}".replace(",", r"\,")

def latex_pct(x: float) -> str:
    return f"{x:.1f}" + r"\%"

def cell(n, denom):
    return f"{latex_int(n)} ({latex_pct(pct(n, denom))})"


In [2]:
def intersect_cylinder_vec_finite(
    exit_x, exit_y, exit_z, nx, ny, nz,
    *, R_det_cm: float, z_max_cm: float, eps: float = 1e-12
):
    """
    Ray intersection with finite cylinder:
      x^2 + y^2 = R_det^2  AND  |z| <= z_max_cm
    Returns px,py,pz (NaN where no valid forward intersection).
    """
    exit_x = np.asarray(exit_x, float)
    exit_y = np.asarray(exit_y, float)
    exit_z = np.asarray(exit_z, float)
    nx = np.asarray(nx, float)
    ny = np.asarray(ny, float)
    nz = np.asarray(nz, float)

    a = nx*nx + ny*ny
    b = 2.0*(exit_x*nx + exit_y*ny)
    c = exit_x*exit_x + exit_y*exit_y - R_det_cm*R_det_cm

    px = np.full_like(exit_x, np.nan, dtype=float)
    py = np.full_like(exit_y, np.nan, dtype=float)
    pz = np.full_like(exit_z, np.nan, dtype=float)

    ok_dir = a > eps
    disc = b*b - 4.0*a*c
    ok = ok_dir & (disc >= 0.0)
    if not np.any(ok):
        return px, py, pz

    idx_ok = np.flatnonzero(ok)
    sqrt_disc = np.sqrt(disc[ok])
    a_ok = a[ok]
    b_ok = b[ok]

    t1 = (-b_ok - sqrt_disc) / (2.0*a_ok)
    t2 = (-b_ok + sqrt_disc) / (2.0*a_ok)

    z1 = exit_z[idx_ok] + t1 * nz[idx_ok]
    z2 = exit_z[idx_ok] + t2 * nz[idx_ok]

    valid1 = (t1 > eps) & (np.abs(z1) <= z_max_cm)
    valid2 = (t2 > eps) & (np.abs(z2) <= z_max_cm)

    t_choice = np.where(valid1 & valid2, np.minimum(t1, t2),
                        np.where(valid1, t1,
                                 np.where(valid2, t2, np.nan)))

    keep = ~np.isnan(t_choice)
    if not np.any(keep):
        return px, py, pz

    idx = idx_ok[keep]
    t_use = t_choice[keep]
    px[idx] = exit_x[idx] + t_use * nx[idx]
    py[idx] = exit_y[idx] + t_use * ny[idx]
    pz[idx] = exit_z[idx] + t_use * nz[idx]
    return px, py, pz


In [3]:
def accumulate_fates_from_photon_shard(
    feather_path: str,
    *,
    R_det_cm: float,
    z_max_det_cm: float,
):
    need = [
        "photon_index",
        "exit_x", "exit_y", "exit_z",
        "n_x", "n_y", "n_z",
        "n_object_scatters",
        "E_exit_keV",
    ]
    opt = ["exited"]   # if present, use it

    df = pd.read_feather(feather_path)
    cols = [c for c in need if c in df.columns] + [c for c in opt if c in df.columns]
    df = df[cols]

    # keep only annihilation photons if there are other photon types (usually photon_index 1/2)
    if "photon_index" in df.columns:
        df = df[df["photon_index"].isin([1, 2])]

    if len(df) == 0:
        return dict(
            exited=0, absorbed=0,
            hit_detector=0, missed_detector=0,
            not_scattered=0, scattered=0
        )

    # exited flag: prefer explicit column; fallback to finite exit energy
    if "exited" in df.columns:
        m_exit = df["exited"].to_numpy(bool)
    else:
        # fallback: treat finite E_exit as "exited"
        m_exit = np.isfinite(df["E_exit_keV"].to_numpy(float))

    n_exit = int(np.sum(m_exit))
    n_abs  = int(len(df) - n_exit)

    if n_exit == 0:
        return dict(
            exited=0, absorbed=n_abs,
            hit_detector=0, missed_detector=0,
            not_scattered=0, scattered=0
        )

    dfe = df.loc[m_exit].copy()

    # scattered vs not scattered (among exited)
    scat = dfe["n_object_scatters"].to_numpy(int)
    n_not_sc = int(np.sum(scat == 0))
    n_sc     = int(np.sum(scat > 0))

    # detector hit among exited
    px, py, pz = intersect_cylinder_vec_finite(
        dfe["exit_x"].to_numpy(float),
        dfe["exit_y"].to_numpy(float),
        dfe["exit_z"].to_numpy(float),
        dfe["n_x"].to_numpy(float),
        dfe["n_y"].to_numpy(float),
        dfe["n_z"].to_numpy(float),
        R_det_cm=R_det_cm,
        z_max_cm=z_max_det_cm,
    )
    hit = np.isfinite(px)  # any valid intersection implies hit
    n_hit = int(np.sum(hit))
    n_miss = int(n_exit - n_hit)

    return dict(
        exited=n_exit,
        absorbed=n_abs,
        hit_detector=n_hit,
        missed_detector=n_miss,
        not_scattered=n_not_sc,
        scattered=n_sc,
    )


In [4]:
stats_by_R = {}

for R in Rs:
    phot_files = sorted(glob.glob(f"{MC_DATA_FOLDER}/photons_*_R{int(R)}.feather"))
    if not phot_files:
        raise FileNotFoundError(f"No photon files found for R={R} in {MC_DATA_FOLDER}")

    tot = dict(
        emitted=TOTAL_NUMBER_OF_PHOTONS,
        exited=0, absorbed=0,
        hit_detector=0, missed_detector=0,
        not_scattered=0, scattered=0,
    )

    for k, f in enumerate(phot_files):
        inc = accumulate_fates_from_photon_shard(
            f,
            R_det_cm=R_DET_CM,
            z_max_det_cm=Z_MAX_DET_CM,
        )
        for key, val in inc.items():
            tot[key] += int(val)

        if (k + 1) % 10 == 0 or (k + 1) == len(phot_files):
            print(f"R={R}: processed {k+1}/{len(phot_files)} shards")

    # sanity checks
    if tot["exited"] + tot["absorbed"] != tot["emitted"]:
        print(f"[WARN] R={R}: exited+absorbed != emitted")

    if tot["hit_detector"] + tot["missed_detector"] != tot["exited"]:
        print(f"[WARN] R={R}: hit+missed != exited")

    if tot["not_scattered"] + tot["scattered"] != tot["exited"]:
        print(f"[WARN] R={R}: not_scattered+scattered != exited")

    stats_by_R[R] = tot

stats_by_R


R=5.0: processed 10/50 shards
R=5.0: processed 20/50 shards
R=5.0: processed 30/50 shards
R=5.0: processed 40/50 shards
R=5.0: processed 50/50 shards
R=10.0: processed 10/50 shards
R=10.0: processed 20/50 shards
R=10.0: processed 30/50 shards
R=10.0: processed 40/50 shards
R=10.0: processed 50/50 shards
R=15.0: processed 10/50 shards
R=15.0: processed 20/50 shards
R=15.0: processed 30/50 shards
R=15.0: processed 40/50 shards
R=15.0: processed 50/50 shards


{5.0: {'emitted': 20000000,
  'exited': 19931171,
  'absorbed': 68829,
  'hit_detector': 10594370,
  'missed_detector': 9336801,
  'not_scattered': 10494265,
  'scattered': 9436906},
 10.0: {'emitted': 20000000,
  'exited': 19380009,
  'absorbed': 619991,
  'hit_detector': 8045072,
  'missed_detector': 11334937,
  'not_scattered': 6130763,
  'scattered': 13249246},
 15.0: {'emitted': 20000000,
  'exited': 18247794,
  'absorbed': 1752206,
  'hit_detector': 3435008,
  'missed_detector': 14812786,
  'not_scattered': 3960534,
  'scattered': 14287260}}

In [5]:
def fmt_int(n: int) -> str:
    return f"{int(n):,}"

def fmt_pct(x: float) -> str:
    return f"{x:.1f}%"

def cell_txt(n, denom):
    return f"{fmt_int(n)} ({fmt_pct(pct(n, denom))})"

cat_names = [
    "Emitted",
    "Absorbed (in object)",
    "Exited object",
    "Exited, not scattered",
    "Exited, scattered",
    "Exited, hit detector",
    "Exited, missed detector",
]

rows = []
for name in cat_names:
    row = {"Category": name}
    for R in Rs:
        s = stats_by_R[R]
        denom = s["emitted"]
        if name == "Emitted":
            row[f"R = {R:.0f} cm"] = f"{fmt_int(s['emitted'])} (100.0%)"
        elif name == "Absorbed (in object)":
            row[f"R = {R:.0f} cm"] = cell_txt(s["absorbed"], denom)
        elif name == "Exited object":
            row[f"R = {R:.0f} cm"] = cell_txt(s["exited"], denom)
        elif name == "Exited, not scattered":
            row[f"R = {R:.0f} cm"] = cell_txt(s["not_scattered"], denom)
        elif name == "Exited, scattered":
            row[f"R = {R:.0f} cm"] = cell_txt(s["scattered"], denom)
        elif name == "Exited, hit detector":
            row[f"R = {R:.0f} cm"] = cell_txt(s["hit_detector"], denom)
        elif name == "Exited, missed detector":
            row[f"R = {R:.0f} cm"] = cell_txt(s["missed_detector"], denom)
    rows.append(row)

df_summary = pd.DataFrame(rows)
df_summary


,Category,R = 5 cm,R = 10 cm,R = 15 cm
0,Emitted,"20,000,000 (100.0%)","20,000,000 (100.0%)","20,000,000 (100.0%)"
1,Absorbed (in object),"68,829 (0.3%)","619,991 (3.1%)","1,752,206 (8.8%)"
2,Exited object,"19,931,171 (99.7%)","19,380,009 (96.9%)","18,247,794 (91.2%)"
3,"Exited, not scattered","10,494,265 (52.5%)","6,130,763 (30.7%)","3,960,534 (19.8%)"
4,"Exited, scattered","9,436,906 (47.2%)","13,249,246 (66.2%)","14,287,260 (71.4%)"
5,"Exited, hit detector","10,594,370 (53.0%)","8,045,072 (40.2%)","3,435,008 (17.2%)"
6,"Exited, missed detector","9,336,801 (46.7%)","11,334,937 (56.7%)","14,812,786 (74.1%)"


In [6]:
cat_names_latex = [
    ("Emitted", "emitted"),
    ("Absorbed", "absorbed"),
    ("Exited", "exited"),
    ("Not scattered (exited)", "not_scattered"),
    ("Scattered (exited)", "scattered"),
    (rf"Hit detector ($R_\mathrm{{det}}={R_DET_CM:.0f}\,\mathrm{{cm}}$, $L={RING_LENGTH_CM:.0f}\,\mathrm{{cm}}$)", "hit_detector"),
    ("Missed detector (exited)", "missed_detector"),
]

header = "Category & " + " & ".join([rf"$R={R:.0f}\,\mathrm{{cm}}$" for R in Rs]) + r" \\"
print(header)
print(r"\midrule")

for label, key in cat_names_latex:
    cells = []
    for R in Rs:
        s = stats_by_R[R]
        denom = s["emitted"]
        if key == "emitted":
            cells.append(f"{latex_int(s['emitted'])} (100.0\\%)")
        else:
            cells.append(cell(s[key], denom))
    print(f"{label} & " + " & ".join(cells) + r" \\")


Category & $R=5\,\mathrm{cm}$ & $R=10\,\mathrm{cm}$ & $R=15\,\mathrm{cm}$ \\
\midrule
Emitted & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) \\
Absorbed & 68\,829 (0.3\%) & 619\,991 (3.1\%) & 1\,752\,206 (8.8\%) \\
Exited & 19\,931\,171 (99.7\%) & 19\,380\,009 (96.9\%) & 18\,247\,794 (91.2\%) \\
Not scattered (exited) & 10\,494\,265 (52.5\%) & 6\,130\,763 (30.7\%) & 3\,960\,534 (19.8\%) \\
Scattered (exited) & 9\,436\,906 (47.2\%) & 13\,249\,246 (66.2\%) & 14\,287\,260 (71.4\%) \\
Hit detector ($R_\mathrm{det}=15\,\mathrm{cm}$, $L=20\,\mathrm{cm}$) & 10\,594\,370 (53.0\%) & 8\,045\,072 (40.2\%) & 3\,435\,008 (17.2\%) \\
Missed detector (exited) & 9\,336\,801 (46.7\%) & 11\,334\,937 (56.7\%) & 14\,812\,786 (74.1\%) \\


In [8]:
def pct(n, denom):
    return 100.0 * n / denom if denom else float("nan")

def latex_int(n: int) -> str:
    return f"{int(n):,}".replace(",", r"\,")

def latex_pct(x: float) -> str:
    return f"{x:.1f}" + r"\%"

def cell_latex(n, denom):
    return f"{latex_int(n)} ({latex_pct(pct(n, denom))})"


In [9]:
# --- TABLE 1: relative to emitted ---
rows1 = [
    ("Emitted", "emitted"),
    ("Absorbed", "absorbed"),
    ("Exited", "exited"),
]

header = "Category & " + " & ".join([rf"$R={R:.0f}\,\mathrm{{cm}}$" for R in Rs]) + r" \\"
print(header)
print(r"\midrule")

for label, key in rows1:
    cells = []
    for R in Rs:
        s = stats_by_R[R]
        denom = s["emitted"]  # 100% = emitted
        if key == "emitted":
            cells.append(f"{latex_int(s['emitted'])} (100.0\\%)")
        else:
            cells.append(cell_latex(s[key], denom))
    print(f"{label} & " + " & ".join(cells) + r" \\")


Category & $R=5\,\mathrm{cm}$ & $R=10\,\mathrm{cm}$ & $R=15\,\mathrm{cm}$ \\
\midrule
Emitted & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) & 20\,000\,000 (100.0\%) \\
Absorbed & 68\,829 (0.3\%) & 619\,991 (3.1\%) & 1\,752\,206 (8.8\%) \\
Exited & 19\,931\,171 (99.7\%) & 19\,380\,009 (96.9\%) & 18\,247\,794 (91.2\%) \\


In [10]:
# --- TABLE 2: relative to exited ---
rows2 = [
    ("Exited", "exited"),
    ("Not scattered", "not_scattered"),
    ("Scattered", "scattered"),
    ("__HLINE__", None),
    (rf"Hit detector ($R_\mathrm{{det}}={R_DET_CM:.0f}\,\mathrm{{cm}}$, $L={RING_LENGTH_CM:.0f}\,\mathrm{{cm}}$)", "hit_detector"),
    ("Missed detector", "missed_detector"),
]

header = "Category & " + " & ".join([rf"$R={R:.0f}\,\mathrm{{cm}}$" for R in Rs]) + r" \\"
print(header)
print(r"\midrule")

for label, key in rows2:
    if label == "__HLINE__":
        print(r"\midrule")
        continue

    cells = []
    for R in Rs:
        s = stats_by_R[R]
        denom = s["exited"]  # 100% = exited
        if key == "exited":
            cells.append(f"{latex_int(s['exited'])} (100.0\\%)")
        else:
            cells.append(cell_latex(s[key], denom))
    print(f"{label} & " + " & ".join(cells) + r" \\")


Category & $R=5\,\mathrm{cm}$ & $R=10\,\mathrm{cm}$ & $R=15\,\mathrm{cm}$ \\
\midrule
Exited & 19\,931\,171 (100.0\%) & 19\,380\,009 (100.0\%) & 18\,247\,794 (100.0\%) \\
Not scattered & 10\,494\,265 (52.7\%) & 6\,130\,763 (31.6\%) & 3\,960\,534 (21.7\%) \\
Scattered & 9\,436\,906 (47.3\%) & 13\,249\,246 (68.4\%) & 14\,287\,260 (78.3\%) \\
\midrule
Hit detector ($R_\mathrm{det}=15\,\mathrm{cm}$, $L=20\,\mathrm{cm}$) & 10\,594\,370 (53.2\%) & 8\,045\,072 (41.5\%) & 3\,435\,008 (18.8\%) \\
Missed detector & 9\,336\,801 (46.8\%) & 11\,334\,937 (58.5\%) & 14\,812\,786 (81.2\%) \\


In [11]:
import pandas as pd

def cell_txt(n, denom):
    return f"{int(n):,} ({pct(n, denom):.1f}%)"

# Table 1 DF
t1 = []
for label, key in rows1:
    row = {"Category": label}
    for R in Rs:
        s = stats_by_R[R]
        denom = s["emitted"]
        if key == "emitted":
            row[f"R={R:.0f} cm"] = f"{s['emitted']:,} (100.0%)"
        else:
            row[f"R={R:.0f} cm"] = cell_txt(s[key], denom)
    t1.append(row)
df1 = pd.DataFrame(t1)

# Table 2 DF
t2 = []
for label, key in rows2:
    if label == "__HLINE__":
        continue
    row = {"Category": label}
    for R in Rs:
        s = stats_by_R[R]
        denom = s["exited"]
        if key == "exited":
            row[f"R={R:.0f} cm"] = f"{s['exited']:,} (100.0%)"
        else:
            row[f"R={R:.0f} cm"] = cell_txt(s[key], denom)
    t2.append(row)
df2 = pd.DataFrame(t2)

display(df1)
display(df2)


,Category,R=5 cm,R=10 cm,R=15 cm
0,Emitted,"20,000,000 (100.0%)","20,000,000 (100.0%)","20,000,000 (100.0%)"
1,Absorbed,"68,829 (0.3%)","619,991 (3.1%)","1,752,206 (8.8%)"
2,Exited,"19,931,171 (99.7%)","19,380,009 (96.9%)","18,247,794 (91.2%)"


,Category,R=5 cm,R=10 cm,R=15 cm
0,Exited,"19,931,171 (100.0%)","19,380,009 (100.0%)","18,247,794 (100.0%)"
1,Not scattered,"10,494,265 (52.7%)","6,130,763 (31.6%)","3,960,534 (21.7%)"
2,Scattered,"9,436,906 (47.3%)","13,249,246 (68.4%)","14,287,260 (78.3%)"
3,"Hit detector ($R_\mathrm{det}=15\,\mathrm{cm}$...","10,594,370 (53.2%)","8,045,072 (41.5%)","3,435,008 (18.8%)"
4,Missed detector,"9,336,801 (46.8%)","11,334,937 (58.5%)","14,812,786 (81.2%)"
